In [ ]:
# https://spark.apache.org/docs/latest/api/python/getting_started/quickstart_df.html

In [ ]:
# 환경변수 설정 필요 

In [3]:
pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [1]:
import findspark

findspark.init()

In [2]:
# spark 생성

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [4]:
# pyspark dataframe 생성1

from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row

df = spark.createDataFrame([
    Row(a=1, b='I', c=date(2010,6,9), d=datetime(2010,6,9,6,9)),
    Row(a=2, b='B', c=date(2013,6,13), d=datetime(2013,6,13,6,13)),
    Row(a=3, b='L', c=date(1999,2,23), d=datetime(1999,2,23, 10,55))
])
df

DataFrame[a: bigint, b: string, c: date, d: timestamp]

In [6]:
# pyspark dataframe 생성2

df = spark.createDataFrame([
    (1, 'I', date(2010,6,9), datetime(2010,6,9,6,9)),
    (2, 'B', date(2013,6,13), datetime(2013,6,13,6,13)),
    (3, 'L', date(1999,2,23), datetime(1999,2,23, 10,55))
], schema='a int, b string, c date, d timestamp')
df

DataFrame[a: int, b: string, c: date, d: timestamp]

In [7]:
# pyspark dataframe 생성3

pandas_df = pd.DataFrame({
    'a': [1, 2, 3],
    'b': ['I','B','L'],
    'c': [date(2010,6,9), date(2013,6,13), date(1999,2,23)],
    'd': [datetime(2010,6,9,6,9), datetime(2013,6,13,6,13), datetime(1999,2,23, 10,55)]
})
df = spark.createDataFrame(pandas_df)
df

DataFrame[a: bigint, b: string, c: date, d: timestamp]

In [8]:
# dataframe 출력

df.show()

+---+---+----------+-------------------+
|  a|  b|         c|                  d|
+---+---+----------+-------------------+
|  1|  I|2010-06-09|2010-06-09 06:09:00|
|  2|  B|2013-06-13|2013-06-13 06:13:00|
|  3|  L|1999-02-23|1999-02-23 10:55:00|
+---+---+----------+-------------------+



In [10]:
# 첫번째 행만 출력하기

df.show(1)

+---+---+----------+-------------------+
|  a|  b|         c|                  d|
+---+---+----------+-------------------+
|  1|  I|2010-06-09|2010-06-09 06:09:00|
+---+---+----------+-------------------+
only showing top 1 row



In [11]:
# 첫번째행 가로로 표시하기

df.show(1, vertical=True)

-RECORD 0------------------
 a   | 1                   
 b   | I                   
 c   | 2010-06-09          
 d   | 2010-06-09 06:09:00 
only showing top 1 row



In [12]:
# 컬럼 출력

df.columns

['a', 'b', 'c', 'd']

In [9]:
# 스키마 출력

df.printSchema()

root
 |-- a: long (nullable = true)
 |-- b: string (nullable = true)
 |-- c: date (nullable = true)
 |-- d: timestamp (nullable = true)



In [13]:
# describe()

df.select('a').describe().show()

+-------+---+
|summary|  a|
+-------+---+
|  count|  3|
|   mean|2.0|
| stddev|1.0|
|    min|  1|
|    max|  3|
+-------+---+



In [14]:
# 분산 데이터 모으기

df.collect()

[Row(a=1, b='I', c=datetime.date(2010, 6, 9), d=datetime.datetime(2010, 6, 9, 6, 9)),
 Row(a=2, b='B', c=datetime.date(2013, 6, 13), d=datetime.datetime(2013, 6, 13, 6, 13)),
 Row(a=3, b='L', c=datetime.date(1999, 2, 23), d=datetime.datetime(1999, 2, 23, 10, 55))]

In [18]:
# 데이터 출력

# head와 같은 역할의 함수

df.take(1)

[Row(a=1, b='I', c=datetime.date(2010, 6, 9), d=datetime.datetime(2010, 6, 9, 6, 9))]

In [19]:
# 데이터 출력

# tail과 같은 역할

df.tail(2)

[Row(a=2, b='B', c=datetime.date(2013, 6, 13), d=datetime.datetime(2013, 6, 13, 6, 13)),
 Row(a=3, b='L', c=datetime.date(1999, 2, 23), d=datetime.datetime(1999, 2, 23, 10, 55))]

In [20]:
# 판다스로 변환

# 판다스api를 사용하고 싶으면 변환

df.toPandas()

,a,b,c,d
0,1,I,2010-06-09,2010-06-09 06:09:00
1,2,B,2013-06-13,2013-06-13 06:13:00
2,3,L,1999-02-23,1999-02-23 10:55:00


In [21]:
df.a

Column<b'a'>

In [24]:
from pyspark.sql import Column
from pyspark.sql.functions import lower

type(df.b) == type(lower(df.b)) == type(df.b.isNull())

True

In [25]:
df.select(df.b).show()

+---+
|  b|
+---+
|  I|
|  B|
|  L|
+---+



In [26]:
# 새로운 컬럼 생성

df.withColumn('lower_b',lower(df.b)).show()

+---+---+----------+-------------------+-------+
|  a|  b|         c|                  d|lower_b|
+---+---+----------+-------------------+-------+
|  1|  I|2010-06-09|2010-06-09 06:09:00|      i|
|  2|  B|2013-06-13|2013-06-13 06:13:00|      b|
|  3|  L|1999-02-23|1999-02-23 10:55:00|      l|
+---+---+----------+-------------------+-------+



In [27]:
# 데이터 필터링

df.filter(df.a==2).show()

+---+---+----------+-------------------+
|  a|  b|         c|                  d|
+---+---+----------+-------------------+
|  2|  B|2013-06-13|2013-06-13 06:13:00|
+---+---+----------+-------------------+



In [3]:
df = spark.createDataFrame([
    ['red', 'banana', 1, 10], ['blue', 'banana', 2, 20], ['red', 'carrot', 3, 30],
    ['blue', 'grape', 4, 40], ['red', 'carrot', 5, 50], ['black', 'carrot', 6, 60],
    ['red', 'banana', 7, 70], ['red', 'grape', 8, 80]], schema=['color', 'fruit', 'v1', 'v2'])
df.show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
|  red|carrot|  5| 50|
|black|carrot|  6| 60|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
+-----+------+---+---+



In [4]:
# groupby

df.groupby('color').avg().show()

+-----+-------+-------+
|color|avg(v1)|avg(v2)|
+-----+-------+-------+
|  red|    4.8|   48.0|
|black|    6.0|   60.0|
| blue|    3.0|   30.0|
+-----+-------+-------+



In [11]:
# 데이터 csv로 저장

# 오류남,,

df.write.csv('foo.csv', header=True)

Py4JJavaError: An error occurred while calling o58.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:178)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at org

In [9]:
df.write.format("csv").save('test2')

Py4JJavaError: An error occurred while calling o50.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:178)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at or

In [8]:
df.write.option("header",True).csv("test")

Py4JJavaError: An error occurred while calling o46.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:178)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at org

In [18]:
# csv 파일 불러오기

spark.read.csv('movies.csv', header=True).show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen